# Preliminaries

In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from scipy import integrate
import cProfile

In [2]:
THIS = 'dissipation-theory--Study-51--'
figs = {}
obj = {}
df = {}

In [3]:
from dissipationtheory.constants import ureg, epsilon0, qe
from dissipationtheory.dissipation9a import CantileverModel, SampleModel1, SampleModel2, SampleModel3, SampleModel4
from dissipationtheory.dissipation9a import integrand1, integrand2, integrand3, isMetal, K, Kunits, Kmetal, Kmetalunits
from dissipationtheory.dissipation9b import CantileverModelJit, SampleModel1Jit, SampleModel2Jit, SampleModel3Jit

# Cantilever

In [4]:
cantilever = CantileverModel(
    f_c = ureg.Quantity(62, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(55, 'nm'),
    angle = ureg.Quantity(20, 'degree'),
    L = ureg.Quantity(1000, 'nm')
)

In [5]:
cantilever

cantilever

         resonance freq = 62.000 kHz
                        = 3.896e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 55.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm

In [6]:
cantilever.args()

{'f_c': 62000.0,
 'k_c': 2.8,
 'V_ts': 1,
 'R': 5.5e-08,
 'angle': 20,
 'L': 1.0000000000000002e-06}

In [7]:
cantilever_jit = CantileverModelJit(**cantilever.args())

In [8]:
cantilever_jit.print()

        cantilever freq =  62000.0 Hz
                        =  389557.48904513434 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  5.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m


# Common reference points

In [9]:
omega = ureg.Quantity(1e5, 'Hz')
loc1 = ureg.Quantity(np.array([ 10, 20, 50]), 'nm')
loc2 = ureg.Quantity(np.array([ 0,   0, 50]), 'nm')

# Type I sample

In [10]:
sample1 = SampleModel1(
    cantilever = cantilever,
    h_s = ureg.Quantity(100, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(100, 'nm')
)

In [11]:
sample1

cantilever

         resonance freq = 62.000 kHz
                        = 3.896e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 55.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm

sample type = 1


semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = 0.000
                  thickness = 100.0 nm
               conductivity = 1.000e-07 S/m
             charge density = 1.000e+21 m^{-3}
           reference height = 1.000e+02 nm

         roll-off frequency = 1.129e+04 Hz
       inverse Debye length = 2.646e-02 nm^{-1}
               Debye length = 3.780e+01 nm
dielectric

  epsilon (real) = 1000000.000
  epsilon (imag) = 0.000
       thickness = infinite

In [12]:
sample1.args()

{'cantilever': <numba.experimental.jitclass.boxing.CantileverModelJit at 0x123c73070>,
 'h_s': 1.0000000000000001e-07,
 'epsilon_s': (20+0j),
 'sigma': 1e-07,
 'rho': 1e+21,
 'epsilon_d': (1000000+0j),
 'z_r': 1.0000000000000001e-07}

In [13]:
sample1_jit = SampleModel1Jit(**sample1.args())

In [14]:
sample1_jit.print()

cantilever
        cantilever freq =  62000.0 Hz
                        =  389557.48904513434 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  5.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m

sample type =  1

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  0.0
               thickness =  1.0000000000000001e-07 m
            conductivity =  1e-07 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  1.0000000000000001e-07 m
 
      roll-off frequency =  11294.09067373019 Hz
    inverse Debye length =  26456583.416667342 m^{{-1}}
            Debye length =  3.7797775481848936e-08 m

dielectric
 epsilon (real) =  1000000.0
 epsilon (imag) =  0.0
      thickness = infinite


In [15]:
K(integrand1, sample1, omega, loc1, loc2)

array([0.92308803-0.00023051j, 0.85642218-0.0003009j ,
       1.57311203-0.00058176j])

In [16]:
Kunits(integrand1, sample1, omega, loc1, loc2)

(<Quantity((0.009230880287373287-2.3050839017765616e-06j), '1 / nanometer')>,
 <Quantity((8.564221806744135e-05-3.008996480189856e-08j), '1 / nanometer ** 2')>,
 <Quantity((1.5731120310679616e-06-5.817587342422955e-10j), '1 / nanometer ** 3')>)

# Type II sample

In [17]:
sample2 = SampleModel2(
    cantilever = cantilever,
    epsilon_d = ureg.Quantity(complex(3, 0), ''),
    h_d = ureg.Quantity(20, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(100, 'nm')
)

In [18]:
sample2

cantilever

         resonance freq = 62.000 kHz
                        = 3.896e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 55.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm

sample type = 2


dielectric

  epsilon (real) = 3.000
  epsilon (imag) = 0.000
       thickness = 20.0 nm

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = 0.000
                  thickness = infinite
               conductivity = 1.000e-07 S/m
             charge density = 1.00e+21 m^{-3}
           reference height = 100.0 nm

         roll-off frequency = 1.129e+04 Hz
       inverse Debye length = 2.646e-02 nm^{-1}
               Debye length = 3.780e+01 nm

In [19]:
sample2.args()

{'cantilever': <numba.experimental.jitclass.boxing.CantileverModelJit at 0x10b647bb0>,
 'epsilon_d': (3+0j),
 'h_d': 2e-08,
 'epsilon_s': (20+0j),
 'sigma': 1e-07,
 'rho': 1e+21,
 'z_r': 1.0000000000000001e-07}

In [20]:
sample2_jit = SampleModel2Jit(**sample2.args())

In [21]:
sample2_jit.print()

cantilever
        cantilever freq =  62000.0 Hz
                        =  389557.48904513434 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  5.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m

sample type =  2

dielectric
 epsilon (real) =  3.0
 epsilon (imag) =  0.0
      thickness =  2e-08 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  0.0
               thickness = infinite
            conductivity =  1e-07 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  1.0000000000000001e-07 m
 
      roll-off frequency =  11294.09067373019 Hz
    inverse Debye length =  26456583.416667342 m^{{-1}}
            Debye length =  3.7797775481848936e-08 m


In [22]:
K(integrand2, sample2, omega, loc1, loc2)

array([0.7921492 -0.0003819j , 0.68749971-0.0002725j ,
       1.18185061-0.00037396j])

In [23]:
Kunits(integrand2, sample2, omega, loc1, loc2)

(<Quantity((0.00792149197984988-3.818985417354145e-06j), '1 / nanometer')>,
 <Quantity((6.874997144531491e-05-2.7249927316501238e-08j), '1 / nanometer ** 2')>,
 <Quantity((1.1818506137568146e-06-3.7396018749568266e-10j), '1 / nanometer ** 3')>)

# Type III sample (semi-infinite semiconductor)

In [24]:
sample3 = SampleModel3(
    cantilever = cantilever,
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(100, 'nm')
)

In [25]:
sample3

cantilever

         resonance freq = 62.000 kHz
                        = 3.896e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 55.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm

sample type = 3

semiconductor

             epsilon (real) = 20.000
             epsilon (imag) = 0.000
                  thickness = infinite
               conductivity = 1.000e-07 S/m
             charge density = 1.00e+21 m^{-3}
           reference height = 100.0 nm

         roll-off frequency = 1.129e+04 Hz
       inverse Debye length = 2.646e-02 nm^{-1}
               Debye length = 3.780e+01 nm

In [26]:
sample3.args()

{'cantilever': <numba.experimental.jitclass.boxing.CantileverModelJit at 0x123e94550>,
 'epsilon_s': (20+0j),
 'sigma': 1e-07,
 'rho': 1e+21,
 'z_r': 1.0000000000000001e-07}

In [27]:
sample3_jit = SampleModel3Jit(**sample3.args())

In [28]:
sample3_jit.print()

cantilever
        cantilever freq =  62000.0 Hz
                        =  389557.48904513434 rad/s
        spring constant =  2.8 N/m
     tip-sample voltage =  1.0 V
                 radius =  5.5e-08 m
        cone half angle =  20.0 degree
            cone length =  1.0000000000000002e-06 m

sample type =  3

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  0.0
               thickness = infinite
            conductivity =  1e-07 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  1.0000000000000001e-07 m
 
      roll-off frequency =  11294.09067373019 Hz
    inverse Debye length =  26456583.416667342 m^{{-1}}
            Debye length =  3.7797775481848936e-08 m


In [29]:
K(integrand3, sample3, omega, loc1, loc2)

array([0.88300038-0.00045539j, 0.84098605-0.00039175j,
       1.56188177-0.00065121j])

In [30]:
Kunits(integrand3, sample3, omega, loc1, loc2)

(<Quantity((0.008830003772773131-4.5539210097441905e-06j), '1 / nanometer')>,
 <Quantity((8.409860514253426e-05-3.91751262512326e-08j), '1 / nanometer ** 2')>,
 <Quantity((1.5618817673292759e-06-6.512113641237762e-10j), '1 / nanometer ** 3')>)

# Type IV sample (metal)

In [31]:
sample4 = SampleModel4(
    cantilever = cantilever,
    z_r = ureg.Quantity(100, 'nm')
)

In [32]:
sample4

cantilever

         resonance freq = 62.000 kHz
                        = 3.896e+05 rad/s
        spring constant = 2.800 N/m
     tip-sample voltage = 1.000 V
                 radius = 55.000 nm
        cone half angle = 20.000 degree
            cone length = 1000.000 nm

sample type = 4

metal

                  thickness = infinite

Run the `isMetal` test on the sample types defined so far.

In [33]:
Kmetal(sample4, loc1, loc2)

((0.9759000729485332-0j), (0.929428640903365+0j), (1.7260817616776778-0j))

In [34]:
Kmetalunits(sample4, loc1, loc2)

(<Quantity((-0.009759000729485332-0j), '1 / nanometer')>,
 <Quantity((9.294286409033649e-05+0j), '1 / nanometer ** 2')>,
 <Quantity((1.7260817616776777e-06-0j), '1 / nanometer ** 3')>)

# Accuracy tests

It's easier to compare the six real values returned by `integrate.quad_vec` than the three complex values returned by `K`.

In [78]:
K(integrand3,sample3, omega, loc1, loc2)

array([0.88300038-0.00045539j, 0.84098605-0.00039175j,
       1.56188177-0.00065121j])

In [79]:
k3 = integrate.quad_vec(integrand3, 0., np.inf, args=(sample3, omega, loc1, loc2))[0]
k3

array([ 8.83000377e-01, -4.55392101e-04,  8.40986051e-01, -3.91751263e-04,
        1.56188177e+00, -6.51211364e-04])

In [56]:
k3u = Kmetalunits(sample3, loc1, loc2)
k3u

(<Quantity((-0.009759000729485332-0j), '1 / nanometer')>,
 <Quantity((9.294286409033649e-05+0j), '1 / nanometer ** 2')>,
 <Quantity((1.7260817616776777e-06-0j), '1 / nanometer ** 3')>)

Below we are going to compare the metal $K_n$'s to the output of `integrate.quad_vec` for Type I, II, and III functions.  The function `Kmetal` returns a complex 3-vector and not a real 6-vector like `integrate.quad_vec`.  Let us define a utilty function that converts a complex 3-vector back into a real 6-vector.

In [57]:
def metalwrapper(x):
    return np.array([x[0].real, x[0].imag, x[1].real, x[1].imag, x[2].real, x[0].imag])

In [58]:
k4 = metalwrapper(Kmetal(sample4, loc1, loc2))
k4

array([ 0.97590007, -0.        ,  0.92942864,  0.        ,  1.72608176,
       -0.        ])

In [59]:
k4u = Kmetalunits(sample4, loc1, loc2)
k4u

(<Quantity((-0.009759000729485332-0j), '1 / nanometer')>,
 <Quantity((9.294286409033649e-05+0j), '1 / nanometer ** 2')>,
 <Quantity((1.7260817616776777e-06-0j), '1 / nanometer ** 3')>)

## Compare Type I and Type III samples

Show that the Type I sample with large $h_{\mathrm{s}}$ ($100 \: \mu\mathrm{m}$) looks like a Type III sample. 

In [39]:
sample1inf = SampleModel1(
    cantilever = cantilever,
    h_s = ureg.Quantity(100, 'um'),
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(100, 'nm')
)

In [40]:
k1inf = integrate.quad_vec(integrand1, 0., np.inf, args=(sample1inf, omega, loc1, loc2))[0]
k1inf

array([ 8.83064933e-01, -4.55052618e-04,  8.40986089e-01, -3.91751066e-04,
        1.56188177e+00, -6.51211364e-04])

The relative errors of the $K_{s}$'s (real and imaginary parts).

In [41]:
(k3 - k1inf)/k3

array([-7.31094703e-05,  7.45474381e-04, -4.49817857e-08,  5.01538941e-07,
       -2.63570691e-11,  3.26016668e-10])

The largest relative errors are in $\text{Im}[K_0]$, $618 \: \mathrm{pp}$ or $0.062 \: \mathrm{percent}$, and $\text{Re}[K_0]$, $-59 \: \mathrm{ppm}$.

## Compare Type II and Type III samples

Show that the Type IU sample with small $h_{\mathrm{d}}$ looks like a Type III sample. 

In [42]:
sample2zero = SampleModel2(
    cantilever = cantilever,
    epsilon_d = ureg.Quantity(complex(3, 0), ''),
    h_d = ureg.Quantity(1e-6, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(100, 'nm')
)

In [43]:
k2zero = integrate.quad_vec(integrand2, 0., np.inf, args=(sample2zero, omega, loc1, loc2))[0]
k2zero

array([ 8.83000372e-01, -4.55392097e-04,  8.40986041e-01, -3.91751257e-04,
        1.56188174e+00, -6.51211350e-04])

The relative errors of the $K_{s}$'s (real and imaginary parts).

In [44]:
(k3 - k2zero)/k3

array([6.22237820e-09, 7.91808152e-09, 1.21337088e-08, 1.52993220e-08,
       1.77099954e-08, 2.21556472e-08])

The largest relative errors are in $\text{Im}[K_2]$, $26 \: \mathrm{ppb}$, and $\text{Re}[K_2]$, $21 \: \mathrm{ppb}$.

## Compare Type I and Type IV samples

Show that the Type I sample with large $h_{\mathrm{s}}$ ($100 \: \mu\mathrm{m}$), $\epsilon_{\mathrm{s}}$, $\sigma$, and $\rho$ looks like a Type IV (metal) sample. 

In [45]:
sample1metal = SampleModel1(
    cantilever = cantilever,
    h_s = ureg.Quantity(100, 'um'),
    epsilon_s = ureg.Quantity(complex(1e6, 0), ''),
    sigma = ureg.Quantity(1e7, 'S/m'),
    rho = ureg.Quantity(1e26, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(100, 'nm')
)

In [46]:
k1metal = integrate.quad_vec(integrand1, 0., np.inf, args=(sample1metal, ureg.Quantity(0, 'Hz'), loc1, loc2))[0]
k1metal

array([0.97589895, 0.        , 0.92942713, 0.        , 1.72607863,
       0.        ])

Only compare the real (even) values since the imaginary (odd) values are zero for a metal.

In [47]:
(k4[0::2] - k1metal[0::2])/k4[0::2]

array([1.14774243e-06, 1.62071636e-06, 1.81366427e-06])

## Compare Type II and Type IV samples

Show that the Type II sample with zero $h_{\mathrm{d}}$ and large $\epsilon_{\mathrm{s}}$, $\sigma$, and $\rho$ looks like a Type IV (metal) sample. 

In [48]:
sample2metal = SampleModel2(
    cantilever = cantilever,
    epsilon_d = ureg.Quantity(complex(3, 0), ''),
    h_d = ureg.Quantity(1e-6, 'nm'),
    epsilon_s = ureg.Quantity(complex(1e6, 0), ''),
    sigma = ureg.Quantity(1e7, 'S/m'),
    rho = ureg.Quantity(1e26, '1/m^3'),
    z_r = ureg.Quantity(100, 'nm')
)

In [49]:
k2metal = integrate.quad_vec(integrand2, 0., np.inf, args=(sample2metal, ureg.Quantity(0, 'Hz'), loc1, loc2))[0]
k2metal

array([0.97589895, 0.        , 0.92942712, 0.        , 1.7260786 ,
       0.        ])

In [50]:
(k4[0::2] - k2metal[0::2])/k4[0::2]

array([1.15409163e-06, 1.63309729e-06, 1.83173506e-06])

## Compare Type III and Type IV samples

Show that the Type III sample with large $\epsilon_{\mathrm{s}}$, $\sigma$, and $\rho$ looks like a Type IV (metal) sample. 

In [51]:
sample3metal = SampleModel3(
    cantilever = cantilever,
    epsilon_s = ureg.Quantity(complex(1e6, 0), ''),
    sigma = ureg.Quantity(1e7, 'S/m'),
    rho = ureg.Quantity(1e26, '1/m^3'),
    z_r = ureg.Quantity(100, 'nm')
)

In [52]:
k3metal = integrate.quad_vec(integrand3, 0., np.inf, args=(sample3metal, ureg.Quantity(0, 'Hz'), loc1, loc2))[0]
k3metal

array([0.97589895, 0.        , 0.92942713, 0.        , 1.72607863,
       0.        ])

In [53]:
(k4[0::2] - k3metal[0::2])/k4[0::2]

array([1.14774243e-06, 1.62071636e-06, 1.81366427e-06])

## Conclusions

The Type II sample with $h_{\mathrm{d}} \rightarrow 0$ does a much better job mimicing the semi-infinite Type III sample than does the Type I sample with $h_{\mathrm{s}} \rightarrow \infty$.

Type I, II, and III samples coverge to a Type IV metallic sample under appropriate sample conditions.

# Additional checks

Make sure that $z_{\mathrm{r}}$ is the same for all samples.

In [56]:
for sample in [sample1, sample2, sample3, sample4, sample1inf, sample1metal]:
    print(sample.z_r)

100 nanometer
100 nanometer
100 nanometer
100 nanometer
100 nanometer
100 nanometer


Can we test complex numbers?  Yes.

In [84]:
K(integrand1, sample1inf, omega, loc1, loc2)

array([0.88306493-0.00045505j, 0.84098609-0.00039175j,
       1.56188177-0.00065121j])

In [85]:
K(integrand3, sample3, omega, loc1, loc2)

array([0.88300038-0.00045539j, 0.84098605-0.00039175j,
       1.56188177-0.00065121j])

In [86]:
np.allclose(K(integrand1, sample1inf, omega, loc1, loc2), 
            K(integrand3, sample3, omega, loc1, loc2), rtol=8e-4)

True

Can we test with units? Yes, with some effort.

In [104]:
temp1 = Kunits(integrand1, sample1inf, omega, loc1, loc2)
temp2 = Kunits(integrand3, sample3, omega, loc1, loc2)

In [110]:
units = ['1/nm','1/nm**2','1/nm**3']
test1 = np.array([val.to(unit).magnitude for val, unit in zip(temp1, units)])
test2 = np.array([val.to(unit).magnitude for val, unit in zip(temp2, units)])

In [111]:
np.allclose(test1, test2, rtol=8e-4)

True

Check the metal $K_n$'s.

In [112]:
Kmetal(sample4, loc1, loc2)

((0.9759000729485332-0j), (0.929428640903365+0j), (1.7260817616776778-0j))

In [113]:
Kmetalunits(sample4, loc1, loc2)

(<Quantity((-0.009759000729485332-0j), '1 / nanometer')>,
 <Quantity((9.294286409033649e-05+0j), '1 / nanometer ** 2')>,
 <Quantity((1.7260817616776777e-06-0j), '1 / nanometer ** 3')>)

# Debug a unit test

In [121]:
K(integrand1, sample1metal, ureg.Quantity(0, 'Hz'), loc1, loc2)

array([0.97589895+0.j, 0.92942713+0.j, 1.72607863+0.j])

In [122]:
Kmetal(sample4, loc1, loc2)

((0.9759000729485332-0j), (0.929428640903365+0j), (1.7260817616776778-0j))

In [123]:
np.allclose(
    K(integrand1, sample1metal, ureg.Quantity(0, 'Hz'), loc1, loc2), 
    Kmetal(sample4, loc1, loc2),
    rtol=2e-6)

True

# Timing tests

## $K_0$ computed using the `v8` code

This is a pain, because the `v8` and `v9` functions are not compatible.

So we have to load in the `v8` version of the functions separately, naming them differently to avoid collisions.

In [60]:
from dissipationtheory.dissipation8a import CantileverModel as CantileverModelv8
from dissipationtheory.dissipation8a import SampleModel1 as SampleModel1v8
from dissipationtheory.dissipation8a import integrand1 as integrand1v8
from dissipationtheory.dissipation8a import K as Kv8

In [61]:
cantileverv8 = CantileverModelv8(
    f_c = ureg.Quantity(62, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(55, 'nm'),
    angle = ureg.Quantity(20, 'degree'),
    L = ureg.Quantity(1000, 'nm'),
    d = ureg.Quantity(50, 'nm')
)

In [62]:
sample1v8 = SampleModel1v8(
    cantilever = cantileverv8,
    h_s = ureg.Quantity(100, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, 0), ''),
    sigma = ureg.Quantity(1e-7, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(100, 'nm')
)

In [63]:
Kv8(integrand1v8, 0, sample1, omega, loc1, loc2, part=np.real).to('1/nm')

<Quantity(0.00923088028737329, '1 / nanometer')>

In [64]:
%%timeit
Kv8(integrand1v8, 0, sample1, omega, loc1, loc2, part=np.real)

453 ms ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## $K_n$'s computed from uncompiled Python functions

Computing the $\text{Re}[K_0]$ value (one number) for a Type I sample using the `v8a` function takes about 800 ms.

Computing the complex $K_0$, $K_1$, and $K_2$ values (six numbers)  for a Type I sample using the `v9a` function takes about 613 ms.  By parallelizing the computations of the $K_n$'s we have sped up the computations by a factor of approximately $(800 \times 6)/1000 = 4.8$.  The running time are quite variable, so this factor is only an estimate.

The time to compute the $K's$ is sample dependent, in this order: Type I > Type II > Type III >> Type IV.

The Python function `Kmetal`, returning three unitless complex $K_{n}$ numbers for the Type IV (metal) sample, takes just 0.4 ms to run.  Blazingly fast, probably because the function uses only (compiled) `numpy` functions.  In contrast, the Python function `K`, doing the same computation for the Type I (semiconductor) sample, takes over 800 ms to run, a factor of 2000 longer!  

Surprisingly, the runtime for `Kmetalunits` is 3.4 ms, nearly a factor of 10 longer than the runtime for `Kmetal`.  Does working with units slow you down?

In [65]:
K(integrand1, sample1, omega, loc1, loc2)

array([0.92308803-0.00023051j, 0.85642218-0.0003009j ,
       1.57311203-0.00058176j])

In [66]:
%%timeit
K(integrand1, sample1, omega, loc1, loc2)

611 ms ± 72.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
Kunits(integrand1, sample1, omega, loc1, loc2)

(<Quantity((0.009230880287373287-2.3050839017765616e-06j), '1 / nanometer')>,
 <Quantity((8.564221806744135e-05-3.008996480189856e-08j), '1 / nanometer ** 2')>,
 <Quantity((1.5731120310679616e-06-5.817587342422955e-10j), '1 / nanometer ** 3')>)

In [68]:
%%timeit
Kunits(integrand1, sample1, omega, loc1, loc2)

578 ms ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
%%timeit
K(integrand2, sample2, omega, loc1, loc2)

328 ms ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%%timeit
Kunits(integrand2, sample2, omega, loc1, loc2)

394 ms ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%%timeit
K(integrand3, sample3, omega, loc1, loc2)

323 ms ± 61.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
%%timeit
Kunits(integrand3, sample3, omega, loc1, loc2)

294 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
Kmetal(sample4, loc1, loc2)

((0.9759000729485332-0j), (0.929428640903365+0j), (1.7260817616776778-0j))

In [74]:
%%timeit
Kmetal(sample4, loc1, loc2)

152 μs ± 18.1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [75]:
Kmetalunits(sample4, loc1, loc2)

(<Quantity((-0.009759000729485332-0j), '1 / nanometer')>,
 <Quantity((9.294286409033649e-05+0j), '1 / nanometer ** 2')>,
 <Quantity((1.7260817616776777e-06-0j), '1 / nanometer ** 3')>)

In [76]:
%%timeit
Kmetalunits(sample4, loc1, loc2)

1.46 ms ± 131 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Why is `Kmetal` a factor of 10 faster than `Kmetalunits`?  Units!  

We can see this in the `cProfile.run()` outputs below.  Many milliseconds is spent on calls to the `quantity.py` package.

In [77]:
cProfile.run("Kmetalunits(sample4, loc1, loc2)",sort='cumulative')

         3702 function calls (3693 primitive calls) in 0.004 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.004    0.004 dissipation9a.py:607(Kmetalunits)
       19    0.000    0.000    0.002    0.000 quantity.py:93(wrapped)
       36    0.000    0.000    0.001    0.000 quantity.py:188(__new__)
        3    0.000    0.000    0.001    0.000 quantity.py:516(to)
       15    0.000    0.000    0.001    0.000 util.py:1031(to_units_container)
        4    0.000    0.000    0.001    0.000 registry.py:59(parse_units_as_container)
        4    0.000    0.000    0.001    0.000 registry.py:1214(parse_units_as_container)
        4    0.000    0.000    0.001    0.000 registry.py:1228(_parse_units_as_container)
        1    0.000    0.000    0.001    0.001 qua

::: {.content-hidden when-format="html"}

# Formatting notes

The header at the top of this file is for creating a nicely-formatted `.html` document using the program `quarto` ([link](https://quarto.org/)).  To create nicely-formated `.html`versions of this notebook, run `quarto` from the command line as follows

    quarto render dissipation-theory--Study-51.ipynb && open dissipation-theory--Study-51.html
    
:::